## Testing of the Graph Environment

Imports

In [1]:
import numpy as np
import pandas as pd
import json
import os
import shutil
import sys
import gym
import pickle
from datetime import datetime

import ray

import warnings
warnings.filterwarnings('ignore')


Import the Environment

In [2]:
sys.path.insert(0,"")

from ManhattanGraph import ManhattanGraph
from gym_graphenv.envs.GraphworldManhattan import GraphEnv

In [3]:
#from ray.rllib.agents.dqn import DQNTrainer, DEFAULT_CONFIG, APEX_DEFAULT_CONFIG
from ray.rllib.agents.ppo import PPOTrainer, DEFAULT_CONFIG

In [4]:
env_config = {'pickup_hub_index':2,
        'delivery_hub_index':3,
        'pickup_timestamp': '2016-01-01 12:00:00', #datetime(2016, 1, 1, 12, 0, 0),
        'delivery_timestamp': '2016-01-01 23:59:00' #datetime(2016, 1, 1, 23, 59, 0) 
}

with open('env_config.pkl', 'wb') as f:
    pickle.dump(env_config, f)

In [5]:
env=GraphEnv()


Run random steps without Ray Trainer

In [6]:
def run_one_episode (env):
    #count_actions = 0
    env.reset()
    print("reset done")
    sum_reward = 0
    for i in range(30):
        action = env.action_space.sample()
        #print(action)
        state, reward, done, info = env.step(action)
        #count_actions += 1
        sum_reward+=reward
        #env.render()
        if done:
            print("DELIVERY DONE! sum_reward: ",sum_reward)
            break

        print("sum_reward: ",sum_reward)
        #print("Number actions: ",count_actions)
        # print("sum_reward: ",sum_reward, " time: ",env.time, "deadline time: ", env.deadline, "pickup time: ", env.pickup_time)
    return sum_reward

for i in range(1): 
    run_one_episode (env)

Reset with config
final hub: 3
current position: 2
DB: getAvailableTrips() Execution time: 49.08999466896057 seconds
Reset() loaded trips within 48 hrs into memory: 20177
in_step_available_trips() Execution time: 0.001991748809814453 seconds
Available rides for share found in step: 0
available trips: []
reset done
Time Ride: 0.002965211868286133
in_step_available_trips() Execution time: 0.0030145645141601562 seconds
Available rides for share found in step: 0
available trips: []
2 -> 4 2201.2459999999996
sum_reward:  -2.201246
Time Ride: 0.05086851119995117
in_step_available_trips() Execution time: 0.00398564338684082 seconds
Available rides for share found in step: 0
available trips: []
4 -> 42 17430.565
sum_reward:  -19.631811
Time Ride: 0.006972789764404297
in_step_available_trips() Execution time: 0.0009970664978027344 seconds
Available rides for share found in step: 0
available trips: []
42 -> 35 5964.152999999999
sum_reward:  -25.595964
Time Ride: 0.014954090118408203
in_step_avai

Initialize Ray

In [7]:
ray.init()

{'node_ip_address': '127.0.0.1',
 'raylet_ip_address': '127.0.0.1',
 'redis_address': None,
 'object_store_address': 'tcp://127.0.0.1:63695',
 'raylet_socket_name': 'tcp://127.0.0.1:64940',
 'webui_url': None,
 'session_dir': 'C:\\Users\\kirch\\AppData\\Local\\Temp\\ray\\session_2022-05-16_12-07-08_660124_18876',
 'metrics_export_port': 63635,
 'gcs_address': '127.0.0.1:64199',
 'address': '127.0.0.1:64199',
 'node_id': '0150057b4e533aaecb7b33008e57a0de2f3cc5f99f98e32414ba938c'}

Set DQN trainer configuration

In [8]:
dqn_config = DEFAULT_CONFIG.copy()
dqn_config['num_workers'] = 1
dqn_config["train_batch_size"] = 400
dqn_config["gamma"] = 0.95
dqn_config["n_step"] = 10
dqn_config["framework"] = "torch"
#num_gpus and other gpu parameters in order to train with gpu
#dqn_config["num_gpus"] = int(os.environ.get("RLLIB_NUM_GPUS", "0")) 

In [8]:
ppo_config = DEFAULT_CONFIG.copy()
ppo_config['framework'] = "torch"

Set DQN - Rainbow configuration

In [9]:
rainbow_config = DEFAULT_CONFIG.copy()

rainbow_config = DEFAULT_CONFIG.copy()
rainbow_config['num_workers'] = 1
rainbow_config["train_batch_size"] = 400
rainbow_config["gamma"] = 0.95
rainbow_config["n_step"] = 10
rainbow_config["framework"] = "torch"
#num_gpus and other gpu parameters in order to train with gpu
#rainbow_config["num_gpus"] = int(os.environ.get("RLLIB_NUM_GPUS", "0")) 

#rainbow parameters

# N-step Q learning
rainbow_config["n_step"]= 5 #[between 1 and 10]
# Whether to use noisy network
rainbow_config["noisy"] = True
# Number of atoms for representing the distribution of return. When
# this is greater than 1, distributional Q-learning is used.
# the discrete supports are bounded by v_min and v_max
rainbow_config["num_atoms"] =2 #[more than 1]
rainbow_config["v_min"] =-100
rainbow_config["v_max"]=100 # (set v_min and v_max according to your expected range of returns).

Set DQN - Ape-X configuration 

In [10]:
#apex_config=APEX_DEFAULT_CONFIG.copy()

Initialize the DQN trainer, Rainbow DQN and DQN Ape-X

In [9]:
ppo_trainer=PPOTrainer(ppo_config,GraphEnv)

2022-05-16 12:07:22,512	INFO ppo.py:249 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
2022-05-16 12:07:22,513	INFO trainer.py:779 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
 pid=25084) UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
 pid=25084) UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
 pid=25084) UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
 pid=25084) UserWarning: The `get_nearest_node` function has been deprecated and will be removed in a future releas

 pid=25084) Reset with config
 pid=25084) final hub: 3
 pid=25084) current position: 2
 pid=17768) Reset with config
 pid=17768) final hub: 3
 pid=17768) current position: 2
 pid=25084) DB: getAvailableTrips() Execution time: 61.89953064918518 seconds
 pid=25084) Reset() loaded trips within 48 hrs into memory: 20177
 pid=17768) DB: getAvailableTrips() Execution time: 61.91945028305054 seconds
 pid=17768) Reset() loaded trips within 48 hrs into memory: 20177
 pid=25084) in_step_available_trips() Execution time: 0.001994609832763672 seconds
 pid=25084) Available rides for share found in step: 0
 pid=25084) available trips: []


2022-05-16 12:09:58,924	INFO trainable.py:127 -- Trainable.setup took 156.416 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2022-05-16 12:09:58,927	WARNING util.py:55 -- Install gputil for GPU system monitoring.


 pid=17768) in_step_available_trips() Execution time: 0.0009970664978027344 seconds
 pid=17768) Available rides for share found in step: 0
 pid=17768) available trips: []


In [ ]:
trainer_dqn = DQNTrainer(dqn_config,GraphEnv)

In [ ]:
trainer_rainbow=DQNTrainer(rainbow_config,GraphEnv )


In [ ]:
#trainer_apex=DQNTrainer(apex_config,GraphEnv )

Define the path where the results of the trainer should be saved

In [10]:
checkpoint_root = "tmp/ppo/graphworld"
shutil.rmtree(checkpoint_root, ignore_errors=True, onerror=None)   # clean up old runs

Run the trainer

In [11]:
results = []
episode_data = []
episode_json = []
n_iter=10

for n in range(n_iter):
    result = ppo_trainer.train()
    results.append(result)
    print("TEST",n)
    
    episode = {'n': n, 
               'episode_reward_min': result['episode_reward_min'], 
               'episode_reward_mean': result['episode_reward_mean'], 
               'episode_reward_max': result['episode_reward_max'],  
               'episode_len_mean': result['episode_len_mean']
              }
    
    episode_data.append(episode)
    episode_json.append(json.dumps(episode))
    file_name = ppo_trainer.save(checkpoint_root)
    
    print(f'{n+1:3d}: Min/Mean/Max reward: {result["episode_reward_min"]:8.4f}/{result["episode_reward_mean"]:8.4f}/{result["episode_reward_max"]:8.4f}, len mean: {result["episode_len_mean"]:8.4f}. Checkpoint saved to {file_name}')

 pid=25084) Reset with config
 pid=25084) final hub: 3
 pid=25084) current position: 2
 pid=17768) Reset with config
 pid=17768) final hub: 3
 pid=17768) current position: 2


KeyboardInterrupt: 

In [ ]:
results

Testing of the trainer

In [ ]:
trainer.restore(file_name)
env = gym.make("graphworld-v0")
state = env.reset()

In [ ]:
sum_reward = 0
n_step = 20
for step in range(n_step):
    action = trainer_dqn.compute_action(state)
    state, reward, done, info = env.step(action)
    sum_reward += reward
    #env.render()
    if done == 1:
        print("cumulative reward", sum_reward)
        state = env.reset()
        sum_reward = 0

In [ ]:
ray.shutdown()